In [49]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [82]:
# import csv file into pandas
# upload to database order (foreign key references require tables be uploaded in ths order): 
# 1: admin_projects
# 2: issues_issue_types
# 3: issues_issue_subtypes
# 4: issues_issues
# note: will be named indicating order

file1 = "data/csv/admin_projects.csv"
file2 = "data/csv/issues_issue_types.csv"
file3 = "data/csv/issues_issue_subtypes.csv"
file4 = "data/csv/issues_issues.csv"
file5 = "static/zip.csv"

csv_data1 = pd.read_csv(file1)
csv_data2 = pd.read_csv(file2)
csv_data3 = pd.read_csv(file3)
csv_data4 = pd.read_csv(file4)
csv_data5 = pd.read_csv(file5, converters={'zip': lambda x: str(x)})

admin_projects_df_1 = pd.DataFrame(csv_data1)
issues_issue_types_df_2 = pd.DataFrame(csv_data2)
issues_issue_subtypes_df_3 = pd.DataFrame(csv_data3)
issues_issues_df_4 = pd.DataFrame(csv_data4)
zipcodes_as_lat_lon = pd.DataFrame(csv_data5)

#confirm databases are loaded
#zipcodes_as_lat_lon.head(5)
admin_projects_df_1.head(5)
#issues_issue_types_df_2.head(5)
#issues_issue_subtypes_df_3.head(5)
#issues_issues_df_4.head(5)





,id,bim360_account_id,name,start_date,end_date,type,value,currency,status,job_number,...,city,state_or_province,postal_code,country,timezone,construction_type,contract_type,business_unit_id,last_sign_in,created_at
0,4fb9e8e5-f9a7-4a04-b33c-38f2227bf75d,49df69ab-c266-4d49-951c-031b16f7a31f,Amanda,Tue May 01 2012 00:00:00 GMT+0000 (Coordinated...,Tue Jan 01 2115 00:00:00 GMT+0000 (Coordinated...,Hospitals & Health Treatment Buildings,750000.0,USD,archived,NaN,...,St Louis,MO,63131,US,America/Chicago,NaN,NaN,NaN,NaN,Thu Aug 27 2015 03:02:51 GMT+0000 (Coordinated...
1,e5ed857f-23f1-4a8c-a808-9cbfa0fe0648,49df69ab-c266-4d49-951c-031b16f7a31f,10100726-UCMC-GARAGE,Tue Aug 23 2016 00:00:00 GMT+0000 (Coordinated...,Tue Aug 24 2021 00:00:00 GMT+0000 (Coordinated...,Office,0.0,USD,archived,NaN,...,NaN,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,Wed Aug 24 2016 01:51:54 GMT+0000 (Coordinated...
2,9d602269-4385-4d46-9d87-ee1374608e23,49df69ab-c266-4d49-951c-031b16f7a31f,10100726-UCMC,Tue Aug 23 2016 00:00:00 GMT+0000 (Coordinated...,Tue Aug 24 2021 00:00:00 GMT+0000 (Coordinated...,Office,0.0,USD,archived,NaN,...,NaN,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,Wed Aug 24 2016 01:51:52 GMT+0000 (Coordinated...
3,003d373a-2b90-45f8-8a0e-862868dfdc0b,49df69ab-c266-4d49-951c-031b16f7a31f,BIM Test Ground,Fri Jan 01 1915 00:00:00 GMT+0000 (Coordinated...,Tue Jan 01 2115 00:00:00 GMT+0000 (Coordinated...,default,0.0,USD,archived,NaN,...,NaN,NaN,63114,US,America/Chicago,NaN,NaN,NaN,NaN,Thu Aug 27 2015 03:03:02 GMT+0000 (Coordinated...
4,9fcfaa9b-b11e-47d9-acbb-a8a8ddf1f7ac,49df69ab-c266-4d49-951c-031b16f7a31f,"Thompson Coburn (ORD), Phase 3",Sun Aug 28 2011 00:00:00 GMT+0000 (Coordinated...,Tue Jan 01 2115 00:00:00 GMT+0000 (Coordinated...,Office & Bank Buildings,0.0,USD,archived,NaN,...,Chicago,IL,60603,US,America/Chicago,NaN,NaN,NaN,NaN,Thu Aug 27 2015 03:03:27 GMT+0000 (Coordinated...


In [ ]:
# Add Data Munging here

In [102]:
clean_df_1 = admin_projects_df_1[admin_projects_df_1["postal_code"].notna()]
clean_df_1 = clean_df_1[clean_df_1["value"] > 5000000]
clean_df_1 = clean_df_1[clean_df_1["status"] == "active"]
clean_df_1.shape

#drop unneeded rows:
clean_df_1 = clean_df_1[['id','bim360_account_id','name','start_date','type','value','currency','status','job_number','city','state_or_province','postal_code','country','timezone','construction_type','contract_type','business_unit_id','last_sign_in','created_at']]
issues_issue_types_df_2 = issues_issue_types_df_2[['issue_type_id','bim360_account_id','bim360_project_id','issue_type','is_active']]
issues_issue_subtypes_df_3 = issues_issue_subtypes_df_3[['issue_subtype_id','bim360_account_id','bim360_project_id','issue_type_id','issue_subtype']]
issues_issues_df_4 = issues_issues_df_4[['issue_id','bim360_project_id','type_id','subtype_id','status','due_date','assignee_id','root_cause_id']]
zipcodes_as_lat_lon = zipcodes_as_lat_lon[['zip','latitude','longitude']] 

clean_df_1_renamed=clean_df_1.rename(columns={"postal_code":"zip"})


#merge tables
zips_and_projects = pd.merge(clean_df_1_renamed, zipcodes_as_lat_lon, on='zip', how="inner")

zips_and_projects.head(5)


,id,bim360_account_id,name,start_date,type,value,currency,status,job_number,city,...,zip,country,timezone,construction_type,contract_type,business_unit_id,last_sign_in,created_at,latitude,longitude
0,e5c196de-27ac-43f1-ad9a-e7267316ed9f,49df69ab-c266-4d49-951c-031b16f7a31f,Memorial Stadium,Mon Jan 01 2018 00:00:00 GMT+0000 (Coordinated...,Stadium/Arena,78000000.0,USD,active,10102939,Champaign,...,61820,US,America/Chicago,New Construction,Design-Bid-Build,1f3be48b-1eef-11e3-89a6-02cc3afa57bd,NaN,Thu Aug 03 2017 16:33:54 GMT+0000 (Coordinated...,40.1018,-88.1972
1,96ab1e3f-d8f3-460e-867e-f28433b7db5a,49df69ab-c266-4d49-951c-031b16f7a31f,10102480 - UIUC Design Center 2,Fri Jun 29 2018 00:00:00 GMT+0000 (Coordinated...,Performing Arts,40000000.0,USD,active,10102480,Champaign,...,61820,US,America/Chicago,New Construction,Design-Bid,03dc88ad-7eb0-48f5-8e12-859300f2a6c8,Mon Feb 10 2020 16:27:07 GMT+0000 (Coordinated...,Mon Jun 25 2018 21:31:43 GMT+0000 (Coordinated...,40.1018,-88.1972
2,b3ed4fac-47bb-43fb-bdd3-3973af5cd19a,49df69ab-c266-4d49-951c-031b16f7a31f,10102939 - Memorial Stadium JV,Wed Mar 01 2017 00:00:00 GMT+0000 (Coordinated...,Recreation Building,90000000.0,USD,active,10102939,Champaign,...,61820,US,America/Chicago,Renovation,Design-Bid-Build,03dc88ad-7eb0-48f5-8e12-859300f2a6c8,Mon Feb 10 2020 15:45:18 GMT+0000 (Coordinated...,Thu Mar 30 2017 14:19:51 GMT+0000 (Coordinated...,40.1018,-88.1972
3,ecee877d-e66c-411c-88ef-3b4f0afbfd1f,49df69ab-c266-4d49-951c-031b16f7a31f,10104945 - Brandon Road Relocation,Sat Feb 01 2020 00:00:00 GMT+0000 (Coordinated...,Streets / Roads / Highways,9000000.0,USD,active,10104945,Joliet,...,60421,US,America/Chicago,New Construction,Design-Bid,869871b3-ac33-4a98-ab47-0ede0ba28d1c,NaN,Mon Feb 03 2020 13:35:47 GMT+0000 (Coordinated...,41.4429,-88.0905
4,800194cf-13d3-47df-9ad1-fe5d2a64d6ca,49df69ab-c266-4d49-951c-031b16f7a31f,10103177 - Rockwool Ran5,Mon Sep 03 2018 00:00:00 GMT+0000 (Coordinated...,Manufacturing / Factory,53362230.0,USD,active,10103177,Shenandoah Junction,...,25442,US,America/New_York,New Construction,Design-Bid,869871b3-ac33-4a98-ab47-0ede0ba28d1c,Mon Feb 10 2020 16:09:36 GMT+0000 (Coordinated...,Tue Jul 31 2018 17:58:10 GMT+0000 (Coordinated...,39.3716,-77.8229


In [85]:
rds_connection_string = "@localhost:5432/project2"

# Important: add password here
password = ""

engine = create_engine(f'postgresql+psycopg2://postgres:{password}{rds_connection_string}')

In [77]:
#confirm database connection and that the tables are present (if not, run schema sql file in database first)
engine.table_names()

['issues_issues',
 'issues_issue_types',
 'admin_projects',
 'issues_issue_subtypes']

In [119]:
#upload data in order specified in beginning


#important note: i removed the foreign key constraints for the time being from the schema for the tables when i
#                ran it in postgres (still in schema file though - at the bottom). Had some issues with FOREIGN
#                KEY CONSTRAINT violations when loading data... missing keys in some tables. so it may be best
#                to run schema without the constraints.


#clean_df_1.to_sql(name='admin_projects', con=engine, if_exists='append', index=False)

#issues_issue_types_df_2.to_sql(name='issues_issue_types', con=engine, if_exists='append', index=False)

#issues_issue_subtypes_df_3.to_sql(name='issues_issue_subtypes', con=engine, if_exists='append', index=False)

#issues_issues_df_4.to_sql(name='issues_issues', con=engine, if_exists='append', index=False)

#zipcodes_as_lat_lon.to_sql(name='zips', con=engine, if_exists='append', index=False)



,issue_type_id,bim360_account_id,bim360_project_id,issue_type,is_active
0,16724ad1-f92a-4ad3-8838-5a66367a60c5,49df69ab-c266-4d49-951c-031b16f7a31f,b082d78a-e1db-4c5c-b06e-eb73fd47a766,Coordination,True
1,e2ca43a8-547d-4b55-b6eb-6e0441a2e3cb,49df69ab-c266-4d49-951c-031b16f7a31f,b082d78a-e1db-4c5c-b06e-eb73fd47a766,Warranty,False
2,d5584757-a911-4910-b7f9-4ae77da54a2a,49df69ab-c266-4d49-951c-031b16f7a31f,b082d78a-e1db-4c5c-b06e-eb73fd47a766,Work List,False
3,b7841417-1299-42ac-868f-f6d2b4aa8fe0,49df69ab-c266-4d49-951c-031b16f7a31f,b082d78a-e1db-4c5c-b06e-eb73fd47a766,Commissioning,True
4,9789f339-0536-45c4-9f93-8a82c92cf518,49df69ab-c266-4d49-951c-031b16f7a31f,b082d78a-e1db-4c5c-b06e-eb73fd47a766,Punch List,True
...,...,...,...,...,...
3019,67b0e535-56de-4a75-a720-970683d1c394,49df69ab-c266-4d49-951c-031b16f7a31f,5ee3dccd-d7cb-4cf0-976b-4a0fe373087f,Work List,False
3020,db9ee598-bd98-48dd-971c-37a1f804cb26,49df69ab-c266-4d49-951c-031b16f7a31f,5ee3dccd-d7cb-4cf0-976b-4a0fe373087f,Commissioning,True
3021,d98679e4-de52-4593-a5ef-c64897c96bf0,49df69ab-c266-4d49-951c-031b16f7a31f,5ee3dccd-d7cb-4cf0-976b-4a0fe373087f,Punch List,True
3022,84a27cee-0a67-4e29-98d4-f1bfc652c1ef,49df69ab-c266-4d49-951c-031b16f7a31f,5ee3dccd-d7cb-4cf0-976b-4a0fe373087f,Safety,True


In [ ]:
from sqlalchemy import Table
from sqlalchemy.ext.compiler import compiles
from sqlalchemy.sql.expression import Executable, ClauseElement

class CreateView(Executable, ClauseElement):
    def __init__(self, name, select):
        self.name = name
        self.select = select

@compiles(CreateView)
def visit_create_view(element, compiler, **kw):
    return "CREATE VIEW %s AS %s" % (
         element.name,
         compiler.process(element.select, literal_binds=True)
         )

# test data
from sqlalchemy import MetaData, Column, Integer


metadata = MetaData(engine)
t = Table('t',
          metadata,
          Column('id', Integer, primary_key=True),
          Column('number', Integer))
t.create()
engine.execute(t.insert().values(id=1, number=3))
engine.execute(t.insert().values(id=9, number=-3))

# create view
createview = CreateView('viewname', t.select().where(t.c.id>5))
engine.execute(createview)

# reflect view and print result
v = Table('viewname', metadata, autoload=True)
for r in engine.execute(v.select()):
    print r

In [123]:
#joins and upload to sql

issues_with_projects = pd.read_sql_query('SELECT admin_projects.id, admin_projects.bim360_account_id, admin_projects.name, admin_projects.start_date, admin_projects.type, admin_projects.value, admin_projects.currency, admin_projects.status AS project_status, admin_projects.job_number, admin_projects.city, admin_projects.state_or_province, admin_projects.postal_code, admin_projects.country, admin_projects.timezone, admin_projects.construction_type, admin_projects.contract_type, admin_projects.business_unit_id, admin_projects.last_sign_in, admin_projects.created_at AS project_created_at, issues_issues.issue_id, issues_issues.bim360_project_id, issues_issues.display_id, issues_issues.title, issues_issues.description, issues_issues.type_id, issues_issues.subtype_id, issues_issues.status AS issue_status, issues_issues.assignee_id, issues_issues.response_by, issues_issues.response_at, issues_issues.opened_by, issues_issues.opened_at, issues_issues.closed_by, issues_issues.closed_at, issues_issues.created_by, issues_issues.created_at AS issue_created_at, issues_issues.updated_by, issues_issues.updated_at FROM admin_projects INNER JOIN issues_issues ON admin_projects.id=issues_issues.bim360_project_id;', con=engine)
issues_with_projects.to_sql(name='issues_with_projects', con=engine, if_exists='append', index=False)
master_file = pd.read_sql_query('SELECT issues_with_projects.id, issues_with_projects.bim360_account_id, issues_with_projects.name, issues_with_projects.start_date, issues_with_projects.type, issues_with_projects.value, issues_with_projects.currency, issues_with_projects.project_status, issues_with_projects.job_number, issues_with_projects.city, issues_with_projects.state_or_province, issues_with_projects.postal_code, issues_with_projects.country, issues_with_projects.timezone, issues_with_projects.construction_type, issues_with_projects.contract_type, issues_with_projects.business_unit_id, issues_with_projects.last_sign_in, issues_with_projects.project_created_at, issues_with_projects.issue_id, issues_with_projects.bim360_project_id, issues_with_projects.display_id, issues_with_projects.title, issues_with_projects.description, issues_with_projects.type_id, issues_with_projects.subtype_id, issues_with_projects.issue_status, issues_with_projects.assignee_id, issues_with_projects.response_by, issues_with_projects.response_at, issues_with_projects.opened_by, issues_with_projects.opened_at, issues_with_projects.closed_by, issues_with_projects.closed_at, issues_with_projects.created_by, issues_with_projects.issue_created_at, issues_with_projects.updated_by, issues_with_projects.updated_at, issues_issue_types.issue_type_id AS issue_type_issue_type_id, issues_issue_types.issue_type AS issue_type_issue_type, issues_issue_types.is_active AS issue_type_is_active,issues_issue_types.created_by AS issue_type_created_by, issues_issue_types.created_at AS issue_type_created_at, issues_issue_types.updated_by AS issue_type_updated_by, issues_issue_types.updated_at AS issue_type_updated_at, issues_issue_types.deleted_by AS issue_type_deleted_by, issues_issue_types.deleted_at AS issue_type_deleted_at FROM issues_with_projects INNER JOIN issues_issue_types ON issues_with_projects.type_id=issues_issue_types.issue_type_id;', con=engine)
master_file.to_sql(name='master_table', con=engine, if_exists='append', index=False)

masterfile_df = pd.read_sql_query('SELECT * FROM master_table', con=engine)
masterfile_df

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "issue_type_issue_type_id" of relation "master_table" does not exist
LINE 1: ...ted_by, issue_created_at, updated_by, updated_at, issue_type...
                                                             ^

[SQL: INSERT INTO master_table (id, bim360_account_id, name, start_date, type, value, currency, project_status, job_number, city, state_or_province, postal_code, country, timezone, construction_type, contract_type, business_unit_id, last_sign_in, project_created_at, issue_id, bim360_project_id, display_id, title, description, type_id, subtype_id, issue_status, assignee_id, response_by, response_at, opened_by, opened_at, closed_by, closed_at, created_by, issue_created_at, updated_by, updated_at, issue_type_issue_type_id, issue_type_issue_type, issue_type_is_active, issue_type_created_by, issue_type_created_at, issue_type_updated_by, issue_type_updated_at, issue_type_deleted_by, issue_type_deleted_at) VALUES (%(id)s, %(bim360

In [117]:
#export to csv file

#import csv
#outfile = open('data/csv/db_to_csv_issues_issue_subtypes.csv', 'wb')
#outcsv = csv.writer(outfile)
#records = engine.query(MyModel).all()
#[outcsv.writerow([getattr(curr, column.name) for column in MyTable.__mapper__.columns]) for curr in records]
# or maybe use outcsv.writerows(records)

#outfile.close()


import csv
#from sqlalchemy.orm import sessionmaker
#Session = sessionmaker(bind = engine)


#session = Session()

#outfile = open('mydump.csv', 'wb')
#outcsv = csv.writer(outfile)

#cursor = engine.execute('select * from admin_projects')

# dump column titles (optional)
#outcsv.writerow(x[0] for x in cursor.description)
# dump rows
#outcsv.writerows(cursor.fetchall())

#outfile.close()



#with open('data/csv/dump.csv', 'w') as f:
#    out = csv.writer(f)
#    out.writerow(['zip', 'latitude', 'longitude', 'city', 'state', 'county', 'other_info'])
#
#    for item in session.query(Queue).all():
#        out.writerow([item.zip, item.latitude, item.longitude, item.city, item.state, item.county, item.other_info])


#dumpfile_df['zip'] = dumpfile_df['zip'].astype('str')
dumpfile_df.to_csv('data/csv/file1.csv')
dumpfile_df


,id,bim360_account_id,name,start_date,type,value,currency,project_status,job_number,city,...,response_by,response_at,opened_by,opened_at,closed_by,closed_at,created_by,issue_created_at,updated_by,updated_at
0,d2310a2c-8320-49b5-b43e-97260b67c88d,49df69ab-c266-4d49-951c-031b16f7a31f,10102944 - SCAD Atlanta,Wed Oct 04 2017 00:00:00 GMT+0000 (Coordinated...,Dormitory,46686936,USD,active,10101748,Atlanta,...,3URYK5ARM6DF,Wed Mar 20 2019 16:17:08 GMT+0000 (Coordinated...,3URYK5ARM6DF,Wed Mar 20 2019 16:16:37 GMT+0000 (Coordinated...,3URYK5ARM6DF,Wed Mar 20 2019 16:17:27 GMT+0000 (Coordinated...,3URYK5ARM6DF,Wed Mar 20 2019 16:16:37 GMT+0000 (Coordinated...,3URYK5ARM6DF,Wed Mar 20 2019 16:17:27 GMT+0000 (Coordinated...
1,328437c3-2ac8-4b4b-847d-0e221871b23e,49df69ab-c266-4d49-951c-031b16f7a31f,10103730 - Centene Hanley Tower TI,Tue Nov 20 2018 00:00:00 GMT+0000 (Coordinated...,Office,34000000,USD,active,10103730,St. Louis,...,9FTXKTWEA7EA,Thu Mar 21 2019 16:31:57 GMT+0000 (Coordinated...,PX6TQPB5HBZM,Tue Mar 05 2019 23:34:36 GMT+0000 (Coordinated...,RSB9Q3EFD7TS,Thu Jul 11 2019 15:31:42 GMT+0000 (Coordinated...,PX6TQPB5HBZM,Tue Mar 05 2019 23:34:36 GMT+0000 (Coordinated...,RSB9Q3EFD7TS,Thu Jul 11 2019 15:31:42 GMT+0000 (Coordinated...
2,8d1b3747-e7f2-4e2d-bc86-e394242c840c,49df69ab-c266-4d49-951c-031b16f7a31f,10103255 - Macy's Chicago,Mon Jun 18 2018 00:00:00 GMT+0000 (Coordinated...,Office,120000000,USD,active,10103255,Chicago,...,TQX5JRYQGMK4,Tue Apr 02 2019 07:26:25 GMT+0000 (Coordinated...,TQX5JRYQGMK4,Tue Apr 02 2019 07:25:27 GMT+0000 (Coordinated...,TQX5JRYQGMK4,Tue Apr 02 2019 07:26:52 GMT+0000 (Coordinated...,TQX5JRYQGMK4,Tue Apr 02 2019 07:25:27 GMT+0000 (Coordinated...,TQX5JRYQGMK4,Tue Apr 02 2019 07:26:52 GMT+0000 (Coordinated...
3,14c8029b-1daf-4fdc-ae19-a5932d354db3,49df69ab-c266-4d49-951c-031b16f7a31f,10103376 - Blue Point,Thu Dec 13 2018 00:00:00 GMT+0000 (Coordinated...,Office,29954578,USD,active,601-03376,Pleasant Prairie,...,None,None,WBKJREULUNNP,Tue Nov 12 2019 14:45:33 GMT+0000 (Coordinated...,6J8SLALCBHQX,Mon Nov 18 2019 16:59:52 GMT+0000 (Coordinated...,WBKJREULUNNP,Tue Nov 12 2019 14:45:33 GMT+0000 (Coordinated...,6J8SLALCBHQX,Mon Nov 18 2019 16:59:52 GMT+0000 (Coordinated...
4,14c8029b-1daf-4fdc-ae19-a5932d354db3,49df69ab-c266-4d49-951c-031b16f7a31f,10103376 - Blue Point,Thu Dec 13 2018 00:00:00 GMT+0000 (Coordinated...,Office,29954578,USD,active,601-03376,Pleasant Prairie,...,None,None,WBKJREULUNNP,Mon Nov 11 2019 18:41:44 GMT+0000 (Coordinated...,WBKJREULUNNP,Tue Nov 12 2019 21:10:28 GMT+0000 (Coordinated...,WBKJREULUNNP,Mon Nov 11 2019 18:41:44 GMT+0000 (Coordinated...,WBKJREULUNNP,Tue Nov 12 2019 21:10:28 GMT+0000 (Coordinated...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4011,8d1b3747-e7f2-4e2d-bc86-e394242c840c,49df69ab-c266-4d49-951c-031b16f7a31f,10103255 - Macy's Chicago,Mon Jun 18 2018 00:00:00 GMT+0000 (Coordinated...,Office,120000000,USD,active,10103255,Chicago,...,6SJ2KWVP6SFS,Mon Feb 03 2020 14:54:35 GMT+0000 (Coordinated...,6SJ2KWVP6SFS,Mon Feb 03 2020 14:54:40 GMT+0000 (Coordinated...,6SJ2KWVP6SFS,Mon Feb 03 2020 14:56:39 GMT+0000 (Coordinated...,6225KHPGAXBM,Sat Feb 01 2020 13:38:33 GMT+0000 (Coordinated...,6SJ2KWVP6SFS,Mon Feb 03 2020 14:56:39 GMT+0000 (Coordinated...
4012,8d1b3747-e7f2-4e2d-bc86-e394242c840c,49df69ab-c266-4d49-951c-031b16f7a31f,10103255 - Macy's Chicago,Mon Jun 18 2018 00:00:00 GMT+0000 (Coordinated...,Office,120000000,USD,active,10103255,Chicago,...,Z72ZP37CU4MT,Mon Feb 03 2020 20:24:37 GMT+0000 (Coordinated...,Z72ZP37CU4MT,Mon Feb 03 2020 20:24:13 GMT+0000 (Coordinated...,Z72ZP37CU4MT,Mon Feb 03 2020 20:25:30 GMT+0000 (Coordinated...,Z72ZP37CU4MT,Mon Feb 03 2020 20:24:13 GMT+0000 (Coordinated...,Z72ZP37CU4MT,Mon Feb 03 2020 20:25:30 GMT+0000 (Coordinated...
4013,8d1b3747-e7f2-4e2d-bc86-e394242c840c,49df69ab-c266-4d49-951c-031b16f7a31f,10103255 - Macy's Chicago,Mon Jun 18 2018 00:00:00 GMT+0000 (Coordinated...,Office,120000000,U

In [33]:
clean_df_1.shape

(197, 22)